# Loading images and folder for preprocessing

In [ ]:
import os
import cv2
import dlib
import numpy as np
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (64, 64))  # Resize
            images.append(img)
    return images
# Load images from drowsy and non-drowsy folders
drowsy_images = load_images_from_folder("dataset/archive/Drowsy/")
non_drowsy_images = load_images_from_folder("dataset/archive/Non Drowsy/")
# Combine the lists
preprocessed_images = drowsy_images + non_drowsy_images

# Loading pre-trained Facial-Detector Model

In [ ]:
predictor_path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Pre-Processing

In [ ]:
import os
import cv2
import dlib
import numpy as np
# Define function to get coordinates of eyes from facial landmarks
def get_eye_landmarks(shape):
    # The dlib facial landmarks detector returns 68 points, the coordinates for left eye are 36 to 41 inclusive
    left_eye = shape[36:42]
    # Coordinates for the right eye are 42 to 47 inclusive
    right_eye = shape[42:48]
    return left_eye, right_eye
# Define function to calculate Eye Aspect Ratio (EAR)
def calculate_ear(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear
# Load dlib's pre-trained facial landmark detector
predictor_path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
# Load and preprocess images
drowsy_folder = "dataset/archive/Drowsy/"
non_drowsy_folder = "dataset/archive/Non Drowsy/"
folders = [drowsy_folder, non_drowsy_folder]
preprocessed_images = []
ear_values = []
for folder in folders:
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (64, 64))  # Resize
            faces = detector(img, 1)
            
            for face in faces:
                shape = predictor(img, face)
                landmarks = np.array([[p.x, p.y] for p in shape.parts()])
                # Get eye landmarks and calculate EAR
                left_eye, right_eye = get_eye_landmarks(landmarks)
                ear_left = calculate_ear(left_eye)
                ear_right = calculate_ear(right_eye)
                ear_avg = (ear_left + ear_right) / 2.0    
                preprocessed_images.append(img)
                ear_values.append(ear_avg)